In [6]:
import pandas as pd
import numpy as np
import pandas_profiling
import re

import matplotlib.pyplot as plt
import seaborn as sns

from fancyimpute import KNN
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from tpot import TPOTRegressor
from auto_ml import Predictor

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
df = pd.read_csv('client-trainingset-1561457457-252.csv')
df.head(2)

,cntryid,cntryid_e,age_r,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,...,v85,v50,v89,v127,v239,v224,v71,v105,row,timestamp
0,Greece,Greece,50.0,Male,Yes,Yes,High,75 to less than 90,NaN,ell,...,Strongly agree,Strongly agree,NaN,NaN,2359.0,9996.0,8549,9996.0,58807,1561457457-252
1,Japan,Japan,47.0,Male,Yes,Yes,High,90 or more,90 or more,jpn,...,Neither agree nor disagree,Neither agree nor disagree,NaN,NaN,2522.0,9996.0,6202,9996.0,168058,1561457457-252


In [3]:
# detected via dataprofiling
correlated_list_1 = ['earnhrbonus', 'earnhrbonusppp', 'earnmthall', 'earnmthallppp', 'earnmthbonus',
                     'earnmthbonusppp', 'earnmthppp', 'isco2l', 'v1', 'v100', 'v110', 'v145', 'v156',
                    'v160', 'v163', 'v169', 'v231', 'v235', 'v283', 'v41', 'v45', 'v52', 'v63', 'v81',
                     'v87', 'v97', 'yrsqual_t']

constant_list_1 = ['timestamp', 'v262']

unsupported_list_1 = ['v71', 'isic2l']

# detected via Comparing information and statistical summary results
correlated_list_2 = ['age_r', 'edcat7', 'edcat8', 'cnt_brth', 'cnt_h', 'cntryid', 'cntryid_e', 'birthrgn',
                     'ctryqual', 'earnhrdcl', 'earnhrbonusdcl', 'nativelang']

# concatenating four lists to drop
drop_list = correlated_list_1 + constant_list_1 + unsupported_list_1 + correlated_list_2

# creating a new dataframe in order to keep original one just in case
df2 = df.copy()

# dropping 43 variables from originial set
df2 = df2.drop(columns = drop_list, axis=1)

# checking summary info
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 337 entries, gender_r to row
dtypes: float64(52), int64(4), object(281)
memory usage: 51.4+ MB


In [4]:
# creating 2 diffrent dataframes as numerical and categorical

categorical_clms = df2.select_dtypes(include=['O']).columns
print("Number of categorical columns: {}".format(len(categorical_clms)))

numerical_clms = df2.select_dtypes(include=['int64', 'float64']).columns
print("Number of numerical columns: {}".format(len(numerical_clms)))

df_cat = pd.DataFrame(df2, columns=categorical_clms)
df_num = pd.DataFrame(df2, columns=numerical_clms)

Number of categorical columns: 281
Number of numerical columns: 56


In [5]:
#dropping target variable
df_num2 = df_num.drop('job_performance', axis=1)

# replacing outliers with median
for feature in df_num2.keys():
    step=1.5*((np.percentile(df_num2[feature], q=75))-(np.percentile(df_num2[feature], q=25)))
    lower_bound=(np.percentile(df_num2[feature], q=25))-step
    upper_bound=(np.percentile(df_num2[feature], q=75))+step
    median = round(df_num2[feature].median())
    df_num2[feature][df_num2[feature]<lower_bound] = median
    df_num2[feature][df_num2[feature]>upper_bound] = median

In [9]:
df_num2.columns.values

array(['yrsqual', 'yrsget', 'imyrs', 'leavedu', 'nfehrsnjr', 'nfehrsjr',
       'nfehrs', 'isco1c', 'isco2c', 'isco1l', 'earnhr', 'earnhrppp',
       'earnmth', 'earnmthselfppp', 'learnatwork', 'readytolearn',
       'icthome', 'ictwork', 'influence', 'planning', 'readhome',
       'readwork', 'taskdisc', 'writhome', 'writwork', 'v202', 'v272',
       'v78', 'v196', 'v61', 'v129', 'v268', 'v252', 'v206', 'v207',
       'v133', 'v136', 'v187', 'v194', 'v154', 'v33', 'v242', 'v184',
       'v104', 'v22', 'v241', 'v135', 'v210', 'v113', 'v130', 'v215',
       'v239', 'v224', 'v105', 'row'], dtype=object)

In [ ]:
df_num_knn = df_num2.copy

df_num_knn = KNN(k=3).fit_transform(df_num_knn)